In [78]:
import pandas as pd
import numpy as np
import seaborn as sns
import shapely.geometry
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt
import os

current_palette = sns.color_palette()
cmap = ListedColormap(sns.color_palette(current_palette).as_hex())

sns.set(rc={'figure.figsize':(10,4)}, font_scale=1.0, style='whitegrid', font='CMU Sans Serif')
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

# this is where we will store all the graphs
graph_folder = os.path.join(".", "graphs")
os.makedirs(graph_folder, exist_ok=True)

def save_fig(ax: plt.Axes, folder: str, name: str, format="pdf"):
    fig = ax.get_figure()
    fig.tight_layout()

    fig.savefig(os.path.join(folder, f"{name}.{format}"), bbox_inches='tight')
    fig.clear()

In [ ]:
# step one: get all the location data
groups = pd.read_csv(os.path.join(".", "groups.csv"))
groups["SENSOR_ID"] = groups["SENSOR_ID"].astype(str)
affinity = pd.read_csv(os.path.join(".", "affinity.csv"))
locations = pd.read_csv("locations.csv")

buoy_locations = locations[locations["type"] == "sensor"]

# to get the links between buoys and sinks, we need to do a few joins here
# first add buoy lat/lng to the groups
groups = groups.merge(locations[["id", "lat", "lng"]], left_on="SENSOR_ID", right_on="id")

# then add the station lat/lng to the affinity
affinity = affinity.merge(locations[["id", "lat", "lng"]], left_on="STATION_ID", right_on="id")

# then merge the affinity with the groups
total_groups = affinity.merge(groups, on="GROUP", suffixes=("_sink", "_buoy"))


In [2]:
# step two: read all the data
experiment_duration = 15 * 60 * 1e3
start_offset = 5 * 60 * 1e3

df_experiment = pd.DataFrame()

for run_name in ["cloud", "sat"]:
    for run_number in ["1", "2", "3"]:

        results_dir = os.path.join(".", "results", f"results-{run_name}-{run_number}")
        files = os.listdir(os.path.join(".", results_dir))

        # first we find out the start time of the experiment
        # the actual start time is the latest "first" entry in all of the files
        start_time = 0.0
        for f in files:
            with open(os.path.join(".", results_dir, f)) as fp:
                # get an iterator
                lines = iter(fp)
                # skip the first line
                next(lines)
                # split the first data line and
                # get send_time_1
                send_time_1 = int(next(lines).split(",")[4])

                if send_time_1 > start_time:
                    start_time = send_time_1

        # convert start_time to milliseconds
        start_time = start_time / 1e6
        start_time += start_offset

        for f in files:

            try:
                client = f[len("cesink"):-len(".ext4-results.csv")]

                df_run = pd.read_csv(os.path.join(".", results_dir, f))
                df_run["client"] = client
                df_run["run_name"] = run_name
                df_run["run_number"] = run_number

                # convert to milliseconds
                df_run["send_time1"] = df_run["send_time1"] / 1e6
                df_run["send_time2"] = df_run["send_time2"] / 1e6
                df_run["recv_time1"] = df_run["recv_time1"] / 1e6
                df_run["recv_time2"] = df_run["recv_time2"] / 1e6

                # now we need to filter out by start_time and duration
                # consider send_time_1 for this
                df_run = df_run[(df_run["send_time1"] >= start_time) & (df_run["send_time1"] <= start_time + experiment_duration)]
                df_run.reset_index(inplace=True)

                df_run["send_latency1"] = df_run["recv_time1"] - df_run["send_time1"]
                df_run["send_latency2"] = df_run["recv_time2"] - df_run["send_time2"]
                df_run["processing_latency"] = df_run["send_time2"] - df_run["recv_time1"]
                df_run["total_latency"] = df_run["recv_time2"] - df_run["send_time1"]

                df_run["latency"] = df_run["recv_time2"] - df_run["send_time1"]

                df_experiment = df_experiment.append(pd.DataFrame({
                    "client": client,
                    "run_name": run_name,
                    "run_number": run_number,
                    "lat": locations.loc[locations["id"] == client, "lat"].values[0],
                    "lng": locations.loc[locations["id"] == client, "lng"].values[0],
                    # at this point we only really care about mean total latency
                    "latency": df_run["total_latency"].mean(),
                }, index=[0]))

            except Exception as e:
                print(f"Error reading file {f}")
                raise e

df_experiment.dropna(inplace=True)
df_experiment.reset_index(inplace=True)


In [106]:
cmap = sns.color_palette("viridis", as_cmap=True)
vmin = 10
vmax = 190

for name in ["cloud", "sat"]:
    for run in ["1", "2", "3"]:
        fig, ax = plt.subplots(figsize=(5, 4), subplot_kw={'projection': ccrs.PlateCarree(central_longitude=180)})

        ax.add_feature(cartopy.feature.BORDERS, linestyle='-', alpha=1, edgecolor=("#FFFFFF"))
        ax.add_feature(cartopy.feature.LAND, facecolor=("#d4d4d4"))

        ax.set_xlim(-60, 110)
        ax.set_ylim(-60, 60)

        ax.grid(False)

        # we need to plot the buoy locations for everyhing to make sense
        sns.scatterplot(ax=ax, data=buoy_locations, x="lng", y="lat", transform=ccrs.Geodetic(), zorder=10, linewidth=0, alpha=0.8, color="black", marker="X", size=0.5)

        # and then add the data
        # just remembered that seaborn doesn't do vmin and vmax properly
        df = df_experiment[(df_experiment["run_name"] == name) & (df_experiment["run_number"] == run)]
        # sns.scatterplot(ax=ax, data=df, x="lng", y="lat",hue="latency", palette=cmap, transform=ccrs.Geodetic(), zorder=10, linewidth=0, alpha=0.8, vmin=vmin, vmax=vmax)
        ax.scatter(x=df["lng"], y=df["lat"], cmap=cmap, c=df["latency"], transform=ccrs.PlateCarree(), zorder=10, linewidth=0, alpha=0.8, vmin=vmin, vmax=vmax)

        # and also the links between things
        for g in range(total_groups.shape[0]):
            ax.plot([total_groups["lng_sink"][g], total_groups["lng_buoy"][g]], [total_groups["lat_sink"][g], total_groups["lat_buoy"][g]], linewidth=0.5, alpha=0.25, color="gray", markersize=1, zorder=10, transform=ccrs.Geodetic())

        # now add a colorbar

        norm = plt.Normalize(vmin, vmax)
        sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
        sm.set_array(np.ndarray([]))
        ax.get_legend().remove()
        cbar = ax.figure.colorbar(sm, location="bottom", shrink=0.35, aspect=10, anchor=(0.85, 1), pad=-0.15)
        cbar.ax.xaxis.set_major_formatter(matplotlib.ticker.FormatStrFormatter('%dms'))
        # custom ticks to get it a bit more spaced out
        cbar.ax.xaxis.set_ticks([25, 100, 175])
        cbar.outline.set_visible(True)
        cbar.outline.set_color("black")
        ax.axis('off')
        # plt.show()
        save_fig(ax, graph_folder, f"buoys-{name}-{run}")
        # ax.clear()

/var/folders/2f/b2y6thfs2051_m0fs57vb_sc0000gn/T/ipykernel_38405/1674366253.py:36: MatplotlibDeprecationWarning: Auto-removal of grids by pcolor() and pcolormesh() is deprecated since 3.5 and will be removed two minor releases later; please call grid(False) first.
  cbar = ax.figure.colorbar(sm, location="bottom", shrink=0.35, aspect=10, anchor=(0.85, 1), pad=-0.15)
webf NOT subset; don't know how to subset; dropped
/var/folders/2f/b2y6thfs2051_m0fs57vb_sc0000gn/T/ipykernel_38405/1674366253.py:36: MatplotlibDeprecationWarning: Auto-removal of grids by pcolor() and pcolormesh() is deprecated since 3.5 and will be removed two minor releases later; please call grid(False) first.
  cbar = ax.figure.colorbar(sm, location="bottom", shrink=0.35, aspect=10, anchor=(0.85, 1), pad=-0.15)
webf NOT subset; don't know how to subset; dropped
/var/folders/2f/b2y6thfs2051_m0fs57vb_sc0000gn/T/ipykernel_38405/1674366253.py:36: MatplotlibDeprecationWarning: Auto-removal of grids by pcolor() and pcolormes

<Figure size 360x288 with 0 Axes>

<Figure size 360x288 with 0 Axes>

<Figure size 360x288 with 0 Axes>

<Figure size 360x288 with 0 Axes>

<Figure size 360x288 with 0 Axes>

<Figure size 360x288 with 0 Axes>